## Import

In [1]:
import warnings

warnings.filterwarnings('ignore')

In [23]:
import random as rnd
import warnings

import matplotlib
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## Reading data:

datapath : "../data"

In [3]:
traindf = pd.read_csv('../data/train.csv', index_col="ID")
traindf.sample(2)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
26728,City Hotel,0,15,2016,February,8,19,0,1,1,0.0,0,BB,PRT,Corporate,TA/TO,0,0,0,A,A,0,No Deposit,NaN,67.0,0,Transient-Party,80.181025,1,0,Check-Out,2016-02-20
6604,City Hotel,0,10,2015,August,36,31,2,5,2,0.0,0,BB,ESP,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,57.0,NaN,0,Transient,72.679909,0,0,Check-Out,2015-09-07


### Drop columns not in test data and Clean missing data
* Drop columns: `is_canceled`, `reservation_status`, `reservation_status_date`, where they aren't in test data

In [4]:
testCol = set(pd.read_csv('../data/test.csv', index_col="ID").columns)
removeCols = [col for col in traindf.columns if col not in testCol]
removeCols.remove("adr")
print(removeCols)

['is_canceled', 'reservation_status', 'reservation_status_date']


In [5]:
traindf = traindf.drop(removeCols, axis=1)
traindf.sample(2)

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22725,Resort Hotel,8,2016,January,3,12,0,2,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,A,D,0,No Deposit,NaN,NaN,0,Transient,31.237071,0,0
30864,Resort Hotel,5,2016,March,13,20,2,2,2,0.0,1,BB,GBR,Offline TA/TO,TA/TO,0,0,0,A,C,1,No Deposit,171.0,NaN,0,Transient,40.916502,0,0


### Missing data
The below code shows column name of missing data

In [6]:
tmp = pd.to_numeric((traindf.isnull().sum() / traindf.shape[0]).map('{:,.2f}'.format))
tmp[tmp > 0]

country    0.01
agent      0.14
company    0.94
dtype: float64

* The company feature's 94% is missing. Because of that, this feature will be eliminated. 
* Another missing data has occurred in country and agent features. Since missing data of country is less than 1%, these data will replace with most frequent value. 
* However, the agent missing features are more than the country. For this feature, missing data will be imputed as 0

In [7]:
# In this state, avoid too many feature, I just drop agent and country columns

traindf["children"] = traindf["children"].fillna(0)
traindf["country"] = traindf["country"].fillna(traindf["country"].mode().index[0])
traindf["agent"] = traindf["agent"].fillna("0")
traindf = traindf.drop(["company", "agent", "country"], axis=1)

In [8]:
traindf.sample(5)

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests
ID,,,,,,,,,,,,,,,,,,,,,,,,,,
62221,City Hotel,0,2016,September,39,20,0,1,2,0.0,0,BB,Direct,Direct,0,0,0,A,C,0,No Deposit,0,Transient,194.683292,0,0
24879,City Hotel,0,2016,February,6,6,0,1,1,0.0,0,BB,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,0,Transient,86.370734,0,0
55432,City Hotel,111,2016,August,33,11,0,2,2,0.0,0,BB,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,0,Transient,118.001497,0,0
68484,Resort Hotel,178,2016,October,43,22,2,2,2,0.0,0,BB,Online TA,TA/TO,0,0,0,E,E,0,No Deposit,0,Transient,63.831011,0,0
18396,City Hotel,9,2015,November,48,22,2,1,1,0.0,0,BB,Corporate,Corporate,1,0,4,A,A,0,No Deposit,0,Transient,90.893819,0,0


### Feature Transformation

`date`: Combine `arrival_date_year`, `arrival_date_month`, `arrival_date_week_number` to date object
<br>One hot encoding `arrival_date_month`

In [9]:
# def create_date(year: pd.Series, month: pd.Series, date: pd.Series) -> pd.Series:
#     return pd.to_datetime(year.astype(str) + month + date.astype(str), format="%Y%B%d")

# traindf.insert(loc=0, column="date", 
#           value=create_date(traindf["arrival_date_year"], traindf["arrival_date_month"], traindf["arrival_date_day_of_month"]))
# traindf = traindf.drop(["arrival_date_year", "arrival_date_month", "arrival_date_day_of_month"], axis=1)

month = pd.get_dummies(traindf.arrival_date_month)
traindf = pd.concat([traindf, month], axis=1, join="inner")
traindf = traindf.drop(["arrival_date_month"], axis=1)

`hotel`: Convert hotel as `Resort Hotel` to 0, `City Hotel` to 1

In [10]:
traindf["hotel"] = traindf["hotel"].map({"Resort Hotel": 0, "City Hotel": 1})

Create new feature `is_same_room`: Check if `reserved_room_type` is same as `assigned_room_type`(If so 1, else 0)
<br>One hot encoding `reserved_room_type` and `assigned_room_type`

In [11]:
traindf["is_same_room"] = (traindf["reserved_room_type"] == traindf["assigned_room_type"]).map({True: 1, False: 0})
reserved = pd.get_dummies(traindf.reserved_room_type, prefix="reserved_")
assigned = pd.get_dummies(traindf.assigned_room_type, prefix="assigned_")
traindf = pd.concat([traindf, reserved, assigned], axis=1, join="inner")
traindf = traindf.drop(["reserved_room_type", "assigned_room_type"], axis=1)

`children`: `children` + `babies`

In [12]:
traindf["children"] = traindf["children"] + traindf["babies"]
traindf = traindf.drop(["babies"], axis=1)

One hot encoding: `meal`

In [13]:
meal = pd.get_dummies(traindf.meal, prefix="meal_")
traindf = pd.concat([traindf, meal], axis=1, join="inner")
traindf = traindf.drop(["meal"], axis=1)

In [14]:
traindf.sample(2)

,hotel,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,April,August,December,February,January,July,June,March,May,November,October,September,is_same_room,reserved__A,reserved__B,reserved__C,reserved__D,reserved__E,reserved__F,reserved__G,reserved__H,reserved__L,reserved__P,assigned__A,assigned__B,assigned__C,assigned__D,assigned__E,assigned__F,assigned__G,assigned__H,assigned__I,assigned__K,assigned__L,assigned__P,meal__BB,meal__FB,meal__HB,meal__SC,meal__Undefined
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
47930,1,8,2016,26,23,0,1,2,0.0,Online TA,TA/TO,0,0,0,0,No Deposit,0,Transient,92.643092,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
65589,0,393,2016,41,8,2,2,2,0.0,Groups,TA/TO,0,0,0,0,Non Refund,0,Transient,54.704680,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


### Train Model and Evaluate best model performance:

Since I only know about regression, I only use SVR as my model, and save the result as 

In [15]:
traindf = pd.get_dummies(traindf)
new_train_x = traindf.drop("adr", axis=1).values
new_train_y = traindf["adr"].values

In [29]:
# Scaler

#from sklearn.preprocessing import MinMaxScaler

#scaler = MinMaxScaler()
#print(scaler.fit(new_train_x))
#print(scaler.transform(new_train_x))
#print(scaler.transform(new_train_y))

In [30]:
X_train, X_test, y_train, y_test = train_test_split(new_train_x, new_train_y, test_size=0.3)
regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

In [33]:
# The result here shows the avg E_in

np.linalg.norm((y_pred - y_test)) / len(y_pred)

0.17806214424870953

In [34]:
regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
regr.fit(new_train_x, new_train_y)
y_pred = regr.predict(new_train_x)

In [35]:
y_pred_csv = pd.DataFrame(y_pred)

In [42]:
y_pred_csv.to_csv("adr_predict.csv", header=["adr"])

In [44]:
adr = pd.DataFrame(new_train_y)

In [45]:
y_pred_csv

,0
0,102.197540
1,60.598775
2,72.030726
3,72.030726
4,72.030726
...,...
91526,96.868011
91527,-2.138195
91528,84.689882
91529,44.351993


In [46]:
adr

,0
0,-6.305161
1,75.052227
2,74.546401
3,76.376288
4,49.411647
...,...
91526,79.223571
91527,-6.822102
91528,90.814554
91529,38.135565


In [47]:
diff = y_pred_csv - adr

In [50]:
diff = diff.apply(abs)

In [57]:
indexes = diff[diff[0] > 50].index

In [61]:
perform_bad = traindf.loc[indexes]

In [77]:
perform_bad[perform_bad["adr"] < 10]["adr"]

0        -6.305161
63       -0.729340
64       -2.902196
119     -88.642804
341      -7.897349
           ...    
91177     7.805044
91261   -62.157892
91263   -59.188184
91275   -68.646270
91305   -44.775831
Name: adr, Length: 1246, dtype: float64

In [78]:
traindf[traindf["adr"] < 10]["adr"]

ID
0        -6.305161
63       -0.729340
64       -2.902196
119     -88.642804
148       8.960203
           ...    
91283   -27.102503
91305   -44.775831
91500   -14.504251
91503   -12.919657
91527    -6.822102
Name: adr, Length: 4010, dtype: float64